- This script reconstructs full-size segmentation masks (12500 * 12500) from U-Net predictions generated from 320×320 crops centered on YOLO-detected centroids. 

- Read YOLO detection results and tile metadata(csv), which are used to map each local centroid back to global image coordinates. For each centroid, the corresponding U-Net prediction (stored in a .json file) is loaded and correctly aligned within the full-size canvas, accounting for tile offsets and boundary clipping. 

- To ensure accurate merging, only valid predicted regions within tile boundaries are pasted, and overlapping regions are resolved using np.maximum() — retaining the highest class index when overlaps occur. <br>

- **Note**
The class index ordering(based on Precision per class) is:
    - 0 = background, 1 = solar panel, 2 = pool heater, 3 = water heater.
- The output is a single .png mask per base image(12500 * 12500 size)

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import json
import re
from tqdm import tqdm

# ============================
# CONFIGURATION
# ============================
yolo_csv_path = "/shared/data/climateplus2025/Prediction_for_poster_July21/processed_centroids_and_bbox.csv"
tile_meta_csv = "/shared/data/climateplus2025/Prediction_for_poster_July21/tile_metadata_for_reconstruction.csv"
# !!! refer to second code chunk in 0.Data_Processing.ipynb in Prediction_for_poster_July21 folder !!!
unet_pred_mask_dir = "/home/cmn60/cape_town_segmentation/prediction_outputs_v42" #v40 : U-Net only, # v42 : YOLO+U-Net # 48 : U-Net only(only predicted value)
# In Catherine's code there are predicted value and Ground Truth (GT) masks together
original_image_shape = (12500, 12500)  # Full image size
output_dir = "/shared/data/climateplus2025/YOLO+U-Net_Prediction_updated_0722/reconstructed_prediction_masks_12500"
tile_crop_size = 320

os.makedirs(output_dir, exist_ok=True)

# ============================
# UTILITY FUNCTIONS
# ============================
def load_json_pred_mask(json_path):
    """
    Reads prediction JSON file and returns a 320x320 class-indexed mask,
    using ONLY 'predicted_coords'. Ground truth data is ignored.
    """
    with open(json_path, "r") as f:
        data = json.load(f)

    pred_coords = data.get("predicted_coords", {})
    mask = np.zeros((tile_crop_size, tile_crop_size), dtype=np.uint8)

    # Class label mapping — must match your model output ordering
    class_order = ["background", "PV_normal", "PV_pool", "PV_heater"]

    for class_idx, cls_name in enumerate(class_order):
        coords = pred_coords.get(cls_name, [])
        for y, x in coords:
            # Protect against malformed coordinates
            if 0 <= y < tile_crop_size and 0 <= x < tile_crop_size:
                mask[y, x] = class_idx

    return mask

# ============================
# LOAD CSVs
# ============================
print("Loading YOLO CSV and tile metadata...")
df = pd.read_csv(yolo_csv_path)
tile_meta_df = pd.read_csv(tile_meta_csv)
tile_meta_df.set_index("tile_name", inplace=True)

# Group by base image
df["base_image_name"] = df["image_name"].apply(lambda x: x.split("_tile_")[0])
grouped = df.groupby("base_image_name")

# ============================
# MAIN PROCESSING LOOP
# ============================
for base_image_name, group_df in grouped:
    print(f"\n▶ Reconstructing mask for: {base_image_name}")

    full_pred_mask = np.zeros(original_image_shape, dtype=np.uint8)

    for idx, row in tqdm(group_df.iterrows(), total=len(group_df)):
        pred_id = row["prediction_id"]
        image_name = row["image_name"]
        centroid_str = row["pixel_centroid"]

        # 1. Load tile metadata
        if image_name not in tile_meta_df.index:
            print(f"[Warning] Missing metadata: {image_name}")
            continue

        tile_info = tile_meta_df.loc[image_name]
        tile_x = int(tile_info["tile_x"])
        tile_y = int(tile_info["tile_y"])
        tile_w = int(tile_info["tile_width"])
        tile_h = int(tile_info["tile_height"])

        # 2. Parse centroid
        try:
            cx, cy = eval(centroid_str)
            cx, cy = int(round(cx)), int(round(cy))
        except:
            print(f"[Warning] Invalid centroid at row {idx}")
            continue

        # 3. Global coordinates of center
        global_cx = tile_x + cx
        global_cy = tile_y + cy

        # 4. Define crop box in global coordinates
        x1 = global_cx - tile_crop_size // 2
        y1 = global_cy - tile_crop_size // 2
        x2 = x1 + tile_crop_size
        y2 = y1 + tile_crop_size

        # 5. Clip to full image boundaries
        x1_clip = max(0, x1)
        y1_clip = max(0, y1)
        x2_clip = min(original_image_shape[1], x2)
        y2_clip = min(original_image_shape[0], y2)

        w = x2_clip - x1_clip
        h = y2_clip - y1_clip
        if w <= 0 or h <= 0:
            continue

        x_offset = x1_clip - x1
        y_offset = y1_clip - y1

        # 6. Load prediction mask
        if not pred_id.startswith("i_"):
            pred_id = "i_" + pred_id

        json_path = os.path.join(unet_pred_mask_dir, f"{pred_id}.json")
        if not os.path.exists(json_path):
            print(f"[Warning] Missing JSON: {json_path}")
            continue

        pred_mask = load_json_pred_mask(json_path)

        # 7. Crop only valid region
        x_crop_end = min(x_offset + w, tile_w)
        y_crop_end = min(y_offset + h, tile_h)
        pred_crop = pred_mask[y_offset:y_crop_end, x_offset:x_crop_end]

        # 8. Paste into full image
        full_pred_mask[y1_clip:y1_clip + pred_crop.shape[0], x1_clip:x1_clip + pred_crop.shape[1]] = np.maximum(
            full_pred_mask[y1_clip:y1_clip + pred_crop.shape[0], x1_clip:x1_clip + pred_crop.shape[1]],
            pred_crop
        )

    # Save result
    output_path = os.path.join(output_dir, f"reconstructed_mask_{base_image_name}.png")
    cv2.imwrite(output_path, full_pred_mask)
    print(f"Saved: {output_path}")


Loading YOLO CSV and tile metadata...

▶ Reconstructing mask for: 2023_RGB_8cm_W24A_17


  0%|          | 0/390 [00:00<?, ?it/s]

100%|██████████| 390/390 [01:12<00:00,  5.38it/s]


Saved: /shared/data/climateplus2025/YOLO+U-Net_Prediction_updated_0722/reconstructed_prediction_masks_12500/reconstructed_mask_2023_RGB_8cm_W24A_17.png

▶ Reconstructing mask for: 2023_RGB_8cm_W25C_16


100%|██████████| 178/178 [00:42<00:00,  4.23it/s]


Saved: /shared/data/climateplus2025/YOLO+U-Net_Prediction_updated_0722/reconstructed_prediction_masks_12500/reconstructed_mask_2023_RGB_8cm_W25C_16.png

▶ Reconstructing mask for: 2023_RGB_8cm_W57B_8


100%|██████████| 242/242 [00:54<00:00,  4.43it/s]


Saved: /shared/data/climateplus2025/YOLO+U-Net_Prediction_updated_0722/reconstructed_prediction_masks_12500/reconstructed_mask_2023_RGB_8cm_W57B_8.png


The output above is a single large prediction mask image (12,500 × 12,500). However, our ground truth (GT) labels were already generated during the U-Net pipeline as cropped 320 × 320 images.

Therefore, we need to crop the large prediction mask into 320 × 320 patches and match them with the corresponding ground truth files.
The file names must match those used in the U-Net ground truth mask folder.
For example: m_xxx_xxx format.

In [5]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# CONFIGURATION
reconstructed_mask_dir = "/shared/data/climateplus2025/YOLO+U-Net_Prediction_updated_0722/reconstructed_prediction_masks"
tile_size = 320
tile_output_dir = "/shared/data/climateplus2025/YOLO+U-Net_Prediction_updated_0722/prediction_masks_tiles_320"

os.makedirs(tile_output_dir, exist_ok=True)

# LOOP OVER MASKS
for filename in tqdm(os.listdir(reconstructed_mask_dir)):
    if not filename.endswith(".png"):
        continue

    base_name = filename.replace("reconstructed_mask_", "").replace(".png", "")
    mask_path = os.path.join(reconstructed_mask_dir, filename)
    mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)  # Read as grayscale or multi-class

    height, width = mask.shape[:2]

    # Calculate padding (if needed)
    pad_h = (tile_size - height % tile_size) % tile_size
    pad_w = (tile_size - width % tile_size) % tile_size
    padded_mask = np.pad(mask, ((0, pad_h), (0, pad_w)), mode='constant')

    padded_height, padded_width = padded_mask.shape

    # Tile loop
    for y in range(0, padded_height, tile_size):
        for x in range(0, padded_width, tile_size):
            tile = padded_mask[y:y+tile_size, x:x+tile_size]

            # Skip blank masks
            if np.all(tile == 0):
                continue

            row_idx = y // tile_size
            col_idx = x // tile_size

            out_filename = f"m_{base_name}_{row_idx}_{col_idx}.png"
            out_path = os.path.join(tile_output_dir, out_filename)

            cv2.imwrite(out_path, tile)


100%|██████████| 3/3 [00:06<00:00,  2.33s/it]
